## Temporal Fusion Transformer

In [21]:
import pandas as pd 
import numpy as np
import torch 
from darts import TimeSeries
from optuna.integration import PyTorchLightningPruningCallback
from pytorch_lightning.callbacks import EarlyStopping

In [22]:
# Import the data 
def load_and_prepare_data(file_path):
    """
    Load energy prices data from a CSV file, ensure chronological order, and convert 'Date' to datetime.
    """
    df = pd.read_csv(file_path)
    df.sort_values('Date', inplace=True)
    # Convert 'date' column to datetime
    df['Date'] = pd.to_datetime(df['Date'])
    #df.set_index('Date', inplace=True)
    df = pd.DataFrame(df)
    return df

In [23]:
# Load in the train and test data
train_df = load_and_prepare_data('../../data/Final_data/train_df.csv')
test_df = load_and_prepare_data('../../data/Final_data/test_df.csv')

# Concatenate the train and test data
df = pd.concat([train_df, test_df])
df['Date'] = pd.to_datetime(df['Date'])

# Create a time series object
series_train = TimeSeries.from_dataframe(train_df, 'Date', 'Day_ahead_price (€/MWh)').astype('float32')
series_test = TimeSeries.from_dataframe(test_df, 'Date', 'Day_ahead_price (€/MWh)').astype('float32')

# Show 
df

,Date,Day_ahead_price (€/MWh),Solar_radiation (W/m2),Wind_speed (m/s),Temperature (°C),Biomass (GWh),Hard_coal (GWh),Hydro (GWh),Lignite (GWh),Natural_gas (GWh),Other (GWh),Pumped_storage_generation (GWh),Solar_energy (GWh),Wind_offshore (GWh),Wind_onshore (GWh),Net_total_export_import (GWh),BEV_vehicles,Oil_price (EUR),TTF_gas_price (€/MWh),Nuclear_energy (GWh)
0,2012-01-01,18.19,14.75,4.95,8.39,98.605,108.454,51.011,325.337,188.811,54.040,19.314,6.263,3.404,235.467,54.662,6,99.64,21.10,250.979
1,2012-01-02,33.82,15.12,5.00,7.41,98.605,222.656,51.862,343.168,229.293,54.166,28.892,6.312,3.350,231.772,-64.477,6,100.04,20.00,258.671
2,2012-01-03,35.03,31.88,7.77,5.23,98.605,162.204,48.851,336.773,241.297,53.518,21.072,24.226,7.292,504.484,-35.078,6,100.44,20.90,271.495
3,2012-01-04,32.16,25.21,8.04,4.78,98.605,189.633,47.101,323.976,252.289,52.194,28.300,14.157,7.828,541.528,22.924,6,103.15,21.40,270.613
4,2012-01-05,20.35,13.46,9.98,4.23,98.605,175.733,45.854,327.502,259.018,52.179,31.887,4.728,8.280,572.819,35.618,6,103.92,21.30,287.555
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
754,2024-07-24,66.61,225.04,3.47,17.54,110.007,43.469,85.857,199.246,194.291,54.026,20.934,325.285,49.360,179.921,-168.705,992,75.75,32.63,0.000
755,2024-07-25,78.34,272.71,2.12,17.85,110.410,50.676,82.632,195.983,209.610,52.963,18.766,394.116,51.053,42.885,-194.496,992,76.36,31.70,0.000
756,2024-07-26,93.04,172.33,2.60,19.09,110.852,42.333,79.531,205.273,205.773,52.616,19.081,256.246,40.449,129.267,-241.786,993,75.21,32.20,0.000
757,2024-07-27,80.74,176.67,2.05,19.63,110.479,33.307,74.958,184.012,216.412,50.927,18.856,244.051,2.180,32.001,-251.655,992,74.79,32.90,0.000


In [24]:
# Define the future covariates columns from your dataframe
future_covariates_columns = ['Solar_radiation (W/m2)', 'Wind_speed (m/s)', 'Temperature (°C)', 
                             'Biomass (GWh)', 'Hard_coal (GWh)', 'Hydro (GWh)', 'Lignite (GWh)', 
                             'Natural_gas (GWh)', 'Other (GWh)', 'Pumped_storage_generation (GWh)', 
                             'Solar_energy (GWh)', 'Wind_offshore (GWh)', 'Wind_onshore (GWh)', 
                             'Net_total_export_import (GWh)', 'BEV_vehicles', 'Oil_price (EUR)', 
                             'TTF_gas_price (€/MWh)', 'Nuclear_energy (GWh)']

## Temporal Fusion Transformer (TFT)

In [25]:
from darts.models import TFTModel
from darts import TimeSeries
from darts.models.forecasting.tft_model import QuantileRegression
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from darts.dataprocessing.transformers import Scaler
from sklearn.preprocessing import MaxAbsScaler, RobustScaler, StandardScaler
from darts.metrics import mape, rmse, mse, mae
import torch

# Set device to MPS if available, otherwise fallback to CPU
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")

# Convert future covariates to TimeSeries objects
future_covariates_train = TimeSeries.from_dataframe(train_df, 'Date', future_covariates_columns).astype('float32')
future_covariates_test = TimeSeries.from_dataframe(test_df, 'Date', future_covariates_columns).astype('float32')

# Convert future covariates to TimeSeries objects
future_covariates_full = TimeSeries.from_dataframe(df, 'Date', future_covariates_columns, fill_missing_dates=True, freq="D").astype('float32')

# Determine required start date for future covariates
input_chunk_length = 200  # Set based on desired look-back period
required_start_date = pd.Timestamp(test_df['Date'].iloc[0]) - pd.DateOffset(days=input_chunk_length)

# Ensure future_covariates_full covers the required range
required_end_date = pd.Timestamp(test_df['Date'].iloc[0]) + pd.DateOffset(days=len(series_test)-1)

# Check if future_covariates_full has sufficient data
if future_covariates_full.start_time() > required_start_date or future_covariates_full.end_time() < required_end_date:
    print("Warning: The future_covariates_full is not long enough to cover the required input chunk length and prediction range.")
    # Extend the future_covariates_full or adjust your dataset

# Slice the future covariates to the required range, including data from the training period
future_covariates_test = future_covariates_full.slice(required_start_date, required_end_date)

# Scaling the data
scaler_series = Scaler(MaxAbsScaler())
scaler_covariates = Scaler(MaxAbsScaler())

# Fit the scaler on the training data
series_train_scaled = scaler_series.fit_transform(series_train)
future_covariates_train_scaled = scaler_covariates.fit_transform(future_covariates_train)

# Transform the test series and future covariates using the same scaler
series_test_scaled = scaler_series.transform(series_test)
future_covariates_test_scaled = scaler_covariates.transform(future_covariates_test)

# Define the TFT model, specifying the use of the MPS device
model = TFTModel(
    input_chunk_length=input_chunk_length,  # History length (number of past time steps used to predict the future)
    output_chunk_length=len(series_test),  # Adjusted to predict the length of the test set
    hidden_size=8,
    lstm_layers=2,
    num_attention_heads=4,
    dropout=0.1,
    likelihood=QuantileRegression(quantiles=[0.1, 0.5, 0.9]),  # Correctly using QuantileRegression
    random_state=42,
    add_relative_index=True,  # Automatically generate relative index as a future covariate
    loss_fn=torch.nn.MSELoss(),  # Correctly using MSELoss
    pl_trainer_kwargs={
        'accelerator': 'gpu',  # Ensure PyTorch Lightning uses GPU
        'devices': 1,  
    }
)

# Train the model with future covariates on MPS
model.fit(series_train_scaled, future_covariates=future_covariates_train_scaled, epochs=5, verbose=True)

# Make predictions on the test set
forecast_scaled = model.predict(n=len(series_test), future_covariates=future_covariates_test_scaled, num_samples=100)

# Inverse transform the forecast to the original scale
forecast = scaler_series.inverse_transform(forecast_scaled)

# Convert TimeSeries to DataFrame for Plotly plotting
test_df_plotly = series_test.pd_dataframe()
forecast_df_plotly = forecast.pd_dataframe()

# Plot the results using Plotly
fig = go.Figure()

# Add actual test data trace
fig.add_trace(go.Scatter(x=test_df_plotly.index, y=test_df_plotly['Day_ahead_price (€/MWh)'],
                         mode='lines', name='Actual Test Data', line=dict(color='darkblue')))

# Add forecast data trace
fig.add_trace(go.Scatter(x=forecast_df_plotly.index, y=forecast_df_plotly['Day_ahead_price (€/MWh)'],
                         mode='lines', name='TFT Model on Test Data', line=dict(color='red')))

# Update layout
fig.update_layout(
    title='TFT Model - Test Performance Only',
    xaxis_title='Date',
    yaxis_title='Day Ahead Price (€/MWh)',
    legend=dict(
        x=1,   # Set x position to 1 (far right)
        y=1,   # Set y position to 1 (top)
        xanchor='right',  # Anchor the legend's x position to the right
        yanchor='top',    # Anchor the legend's y position to the top
        bordercolor='black',  # Optional: Add a border around the legend
        borderwidth=1        # Optional: Set the border width
    ),
    template='plotly'
)

# Show the plot
fig.show()


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

   | Name                              | Type                             | Params
----------------------------------------------------------------------------------------
0  | criterion                         | MSELoss                          | 0     
1  | train_criterion                   | MSELoss                          | 0     
2  | val_criterion                     | MSELoss                          | 0     
3  | train_metrics                     | MetricCollection                 | 0     
4  | val_metrics                       | MetricCollection                 | 0     
5  | input_embeddings                  | _MultiEmbedding                  | 0     
6  | static_covariates_vsn             | _VariableSelectionNetwork        | 0     
7  | encoder_vsn                       | _VariableSelectionNetwork        | 8.3 K 
8  | decoder

Training: 0it [00:00, ?it/s]

In [20]:
# Evaluate the model using Darts' metrics
print(f'Mean Absolute Error on Test Set: {mae(series_test, forecast)}')
print(f'Mean Absolute Percentage Error on Test Set: {mape(series_test, forecast)}')
print(f'Mean Squared Error on Test Set: {mse(series_test, forecast)}')
print(f'Root Mean Squared Error on Test Set: {rmse(series_test, forecast)}')

Mean Absolute Error on Test Set: nan
Mean Absolute Percentage Error on Test Set: nan
Mean Squared Error on Test Set: nan
Root Mean Squared Error on Test Set: nan


/Users/skyfano/Documents/Masterarbeit/Prediction_of_energy_prices/master_thesis/lib/python3.10/site-packages/darts/metrics/metrics.py:785: RuntimeWarning:

Mean of empty slice

/Users/skyfano/Documents/Masterarbeit/Prediction_of_energy_prices/master_thesis/lib/python3.10/site-packages/darts/metrics/metrics.py:244: RuntimeWarning:

Mean of empty slice

/Users/skyfano/Documents/Masterarbeit/Prediction_of_energy_prices/master_thesis/lib/python3.10/site-packages/darts/metrics/metrics.py:1904: RuntimeWarning:

Mean of empty slice

/Users/skyfano/Documents/Masterarbeit/Prediction_of_energy_prices/master_thesis/lib/python3.10/site-packages/darts/metrics/metrics.py:1161: RuntimeWarning:

Mean of empty slice



In [8]:
# Save the created figure as png file and the error metrics 
fig.write_image("../../predictions/TFT/TFT_epochs_10.png")
error_metrics = pd.DataFrame({'MAE': [mae(series_test, forecast)], 'MAPE': [mape(series_test, forecast)], 'MSE': [mse(series_test, forecast)], 'RMSE': [rmse(series_test, forecast)]})
error_metrics.to_csv('../../predictions/TFT/TFT_metrics_epochs_10.csv', index=False)


## Für 30 epochs

In [ ]:
from darts.models import TFTModel
from darts import TimeSeries
from darts.models.forecasting.tft_model import QuantileRegression
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from darts.dataprocessing.transformers import Scaler
from sklearn.preprocessing import MaxAbsScaler, RobustScaler, StandardScaler
from darts.metrics import mape, rmse, mse, mae

# Convert future covariates to TimeSeries objects
future_covariates_train = TimeSeries.from_dataframe(train_df, 'Date', future_covariates_columns).astype('float32')
future_covariates_test = TimeSeries.from_dataframe(test_df, 'Date', future_covariates_columns).astype('float32')

future_covariates_full = TimeSeries.from_dataframe(df, 'Date', future_covariates_columns, fill_missing_dates=True, freq="D").astype('float32')

# Determine required start date for future covariates
input_chunk_length = 200  # Set based on desired look-back period
required_start_date = pd.Timestamp(test_df['Date'].iloc[0]) - pd.DateOffset(days=input_chunk_length)

# Ensure future_covariates_full covers the required range
required_end_date = pd.Timestamp(test_df['Date'].iloc[0]) + pd.DateOffset(days=len(series_test)-1)

# Check if future_covariates_full has sufficient data
if future_covariates_full.start_time() > required_start_date or future_covariates_full.end_time() < required_end_date:
    print("Warning: The future_covariates_full is not long enough to cover the required input chunk length and prediction range.")
    # Extend the future_covariates_full or adjust your dataset

# Slice the future covariates to the required range, including data from the training period
future_covariates_test = future_covariates_full.slice(required_start_date, required_end_date)

# Scaling the data
scaler_series = Scaler(MaxAbsScaler())
scaler_covariates = Scaler(MaxAbsScaler())

# Fit the scaler on the training data
series_train_scaled = scaler_series.fit_transform(series_train)
future_covariates_train_scaled = scaler_covariates.fit_transform(future_covariates_train)

# Transform the test series and future covariates using the same scaler
series_test_scaled = scaler_series.transform(series_test)
future_covariates_test_scaled = scaler_covariates.transform(future_covariates_test)

# Define the TFT model
model = TFTModel(
    input_chunk_length=input_chunk_length,  # History length (number of past time steps used to predict the future)
    output_chunk_length=len(series_test),  # Adjusted to predict the length of the test set
    hidden_size=16,
    lstm_layers=4,
    num_attention_heads=4,
    dropout=0.1,
    likelihood=QuantileRegression(quantiles=[0.1, 0.5, 0.9]),  # Correctly using QuantileRegression
    random_state=42,
    add_relative_index=True,  # Automatically generate relative index as a future covariate
    loss_fn=torch.nn.MSELoss()  # Correctly using MSELoss
)

# Train the model with future covariates
model.fit(series_train_scaled, future_covariates=future_covariates_train_scaled, epochs=30, verbose=True)

# Make predictions on the test set
forecast_scaled = model.predict(n=len(series_test), future_covariates=future_covariates_test_scaled)

# Inverse transform the forecast to the original scale
forecast = scaler_series.inverse_transform(forecast_scaled)

# Convert TimeSeries to DataFrame for Plotly plotting
test_df_plotly = series_test.pd_dataframe()
forecast_df_plotly = forecast.pd_dataframe()

# Plot the results using Plotly
fig_2 = go.Figure()

# Add actual test data trace
fig_2.add_trace(go.Scatter(x=test_df_plotly.index, y=test_df_plotly['Day_ahead_price (€/MWh)'],
                         mode='lines', name='Actual Test Data', line=dict(color='darkblue')))

# Add forecast data trace
fig_2.add_trace(go.Scatter(x=forecast_df_plotly.index, y=forecast_df_plotly['Day_ahead_price (€/MWh)'],
                         mode='lines', name='TFT Model on Test Data', line=dict(color='red')))

# Update layout
fig_2.update_layout(
    title='TFT Model - Test Performance Only',
    xaxis_title='Date',
    yaxis_title='Day Ahead Price (€/MWh)',
    legend=dict(
        x=1,   # Set x position to 1 (far right)
        y=1,   # Set y position to 1 (top)
        xanchor='right',  # Anchor the legend's x position to the right
        yanchor='top',    # Anchor the legend's y position to the top
        bordercolor='black',  # Optional: Add a border around the legend
        borderwidth=1        # Optional: Set the border width
    ),
    template='plotly'
)

# Show the plot
fig_2.show()

In [ ]:
# Evaluate the model using Darts' metrics
print(f'Mean Absolute Error on Test Set: {mae(series_test, forecast)}')
print(f'Mean Absolute Percentage Error on Test Set: {mape(series_test, forecast)}')
print(f'Mean Squared Error on Test Set: {mse(series_test, forecast)}')
print(f'Root Mean Squared Error on Test Set: {rmse(series_test, forecast)}')

In [ ]:
# Save the created figure as png file and the error metrics 
fig_2.write_image("../../predictions/Other_Ideas/TFT_epochs_30.png")
error_metrics = pd.DataFrame({'MAE': [mae(series_test, forecast)], 'MAPE': [mape(series_test, forecast)], 'MSE': [mse(series_test, forecast)], 'RMSE': [rmse(series_test, forecast)]})
error_metrics.to_csv('../../predictions/Other_Ideas/TFT_metrics_epochs_30.csv', index=False)


### Hyperparameter Optimiziation 

In [31]:
import optuna
from darts.models import TFTModel
from darts import TimeSeries
from darts.models.forecasting.tft_model import QuantileRegression
import pandas as pd
import numpy as np
import torch
import plotly.graph_objs as go
from darts.dataprocessing.transformers import Scaler
from darts.metrics import mape, rmse, mse, mae

# Fixed input chunk length
fixed_input_chunk_length = 200  # You can change this value based on your preference.

# Convert future covariates to TimeSeries objects
future_covariates_train = TimeSeries.from_dataframe(train_df, 'Date', future_covariates_columns).astype('float32')

# Convert full covariates to TimeSeries objects
future_covariates_full = TimeSeries.from_dataframe(df, 'Date', future_covariates_columns, fill_missing_dates=True, freq="D").astype('float32')

# Scaling the data
scaler_series = Scaler()
scaler_covariates = Scaler()

# Fit the scaler on the training data
series_train_scaled = scaler_series.fit_transform(series_train)
future_covariates_train_scaled = scaler_covariates.fit_transform(future_covariates_train)

# Transform the test series using the same scaler
series_test_scaled = scaler_series.transform(series_test)

# Recalculate required start and end dates based on fixed input_chunk_length
required_start_date = pd.Timestamp(test_df['Date'].iloc[0]) - pd.DateOffset(days=fixed_input_chunk_length)
required_end_date = pd.Timestamp(test_df['Date'].iloc[0]) + pd.DateOffset(days=len(series_test)-1)

# Check if future_covariates_full has sufficient data
if future_covariates_full.start_time() > required_start_date or future_covariates_full.end_time() < required_end_date:
    print(f"Warning: The future_covariates_full is not long enough for input_chunk_length = {fixed_input_chunk_length}")

# Slice and scale the future covariates
# Slice future_covariates_test to match the exact length of series_test
future_covariates_test = future_covariates_test.slice(required_start_date, required_end_date)
future_covariates_test_scaled = scaler_covariates.transform(future_covariates_test)

def objective(trial):
    # Suggest hyperparameters (excluding input_chunk_length since it's fixed)
    hidden_size = trial.suggest_int('hidden_size', 8, 64)
    lstm_layers = trial.suggest_int('lstm_layers', 1, 4)
    num_attention_heads = trial.suggest_int('num_attention_heads', 1, 8)
    dropout = trial.suggest_float('dropout', 0.0, 0.5)

    # Initialize the TFT model with suggested hyperparameters
    model = TFTModel(
        input_chunk_length=fixed_input_chunk_length,
        output_chunk_length=7,
        hidden_size=hidden_size,
        lstm_layers=lstm_layers,
        num_attention_heads=num_attention_heads,
        dropout=dropout,
        likelihood=QuantileRegression(quantiles=[0.1, 0.5, 0.9]),
        random_state=42,
        add_relative_index=True
    )

    try:
        # Train the model
        model.fit(series_train_scaled, future_covariates=future_covariates_train_scaled, epochs=1, verbose=False)

        # Make predictions
        forecast_scaled = model.predict(n=len(series_test_scaled), future_covariates=future_covariates_test_scaled)

        # Inverse transform the forecast to the original scale
        forecast = scaler_series.inverse_transform(forecast_scaled)

        # Return Mean Squared Error as the objective metric for Optuna to minimize
        error = mse(series_test, forecast)
        if np.isnan(error):
            return float('inf')
        return error
    except Exception as e:
        print(f"Trial failed due to: {e}")
        return float('inf')

# Run the Optuna study with more trials, using the fixed input_chunk_length
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=5, n_jobs=1)

# Print the best hyperparameters
print('Best hyperparameters: ', study.best_params)

# Use the best hyperparameters with the fixed input_chunk_length
best_params = study.best_params

# Initialize the final model with best hyperparameters
best_model = TFTModel(
    input_chunk_length=fixed_input_chunk_length,
    output_chunk_length=7,
    hidden_size=best_params['hidden_size'],
    lstm_layers=best_params['lstm_layers'],
    num_attention_heads=best_params['num_attention_heads'],
    dropout=best_params['dropout'],
    likelihood=QuantileRegression(quantiles=[0.1, 0.5, 0.9]),
    random_state=42,
    add_relative_index=True,
    loss_fn=torch.nn.MSELoss()
)

# Train the best model
best_model.fit(series_train_scaled, future_covariates=future_covariates_train_scaled, epochs=1, verbose=True)

# Make predictions starting from the correct point
forecast_scaled = best_model.predict(
    n=len(series_test_scaled),
    future_covariates=future_covariates_test_scaled
)

# Inverse transform the forecast to the original scale
forecast = scaler_series.inverse_transform(forecast_scaled)

# Convert TimeSeries to DataFrame for Plotly plotting
test_df_plotly = series_test.pd_dataframe()
forecast_df_plotly = forecast.pd_dataframe()

# Plot the results using Plotly
fig = go.Figure()

# Add actual test data trace
fig.add_trace(go.Scatter(x=test_df_plotly.index, y=test_df_plotly['Day_ahead_price (€/MWh)'],
                         mode='lines', name='Actual Test Data', line=dict(color='darkblue')))

# Add forecast data trace
fig.add_trace(go.Scatter(x=forecast_df_plotly.index, y=forecast_df_plotly['Day_ahead_price (€/MWh)'],
                         mode='lines', name='TFT Forecast on Test Data', line=dict(color='red')))

# Update layout
fig.update_layout(
    title='TFT Model - Test Performance with Optuna Hyperparameter Tuning',
    xaxis_title='Date',
    yaxis_title='Day Ahead Price (€/MWh)',
    legend=dict(
        x=1,
        y=1,
        xanchor='right',
        yanchor='top',
        bordercolor='black',
        borderwidth=1
    ),
    template='plotly'
)

# Show the plot
fig.show()

# Evaluate the model using Darts' metrics
print(f'Mean Absolute Error on Test Set: {mae(series_test, forecast)}')
print(f'Mean Absolute Percentage Error on Test Set: {mape(series_test, forecast)}')
print(f'Mean Squared Error on Test Set: {mse(series_test, forecast)}')
print(f'Root Mean Squared Error on Test Set: {rmse(series_test, forecast)}')


[I 2024-09-25 11:04:28,257] A new study created in memory with name: no-name-9fa8eb93-c6ed-4f69-9ae8-6595d23e1cb6
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer.fit` stopped: `max_epochs=1` reached.
ValueError: For the given forecasting horizon `n=759`, the provided future covariates at dataset index `0` do not extend far enough into the future. As `n > output_chunk_length` the future covariates must end at time step `2024-07-28 00:00:00`, whereas now they end at time step `2024-01-10 00:00:00`.
[I 2024-09-25 11:05:11,227] Trial 0 finished with value: inf and parameters: {'hidden_size': 47, 'lstm_layers': 1, 'num_attention_heads': 1, 'dropout': 0.38824819536107896}. Best is trial 0 with value: inf.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Trial failed due to: For the given forecasting horizon `n=759`, the provided future covariates at dataset index `0` do not extend far enough into the future. As `n > output_chunk_length` the future covariates must end at time step `2024-07-28 00:00:00`, whereas now they end at time step `2024-01-10 00:00:00`.


`Trainer.fit` stopped: `max_epochs=1` reached.
ValueError: For the given forecasting horizon `n=759`, the provided future covariates at dataset index `0` do not extend far enough into the future. As `n > output_chunk_length` the future covariates must end at time step `2024-07-28 00:00:00`, whereas now they end at time step `2024-01-10 00:00:00`.
[I 2024-09-25 11:05:43,507] Trial 1 finished with value: inf and parameters: {'hidden_size': 8, 'lstm_layers': 3, 'num_attention_heads': 7, 'dropout': 0.20709801440892822}. Best is trial 0 with value: inf.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Trial failed due to: For the given forecasting horizon `n=759`, the provided future covariates at dataset index `0` do not extend far enough into the future. As `n > output_chunk_length` the future covariates must end at time step `2024-07-28 00:00:00`, whereas now they end at time step `2024-01-10 00:00:00`.


`Trainer.fit` stopped: `max_epochs=1` reached.
ValueError: For the given forecasting horizon `n=759`, the provided future covariates at dataset index `0` do not extend far enough into the future. As `n > output_chunk_length` the future covariates must end at time step `2024-07-28 00:00:00`, whereas now they end at time step `2024-01-10 00:00:00`.
[I 2024-09-25 11:06:26,816] Trial 2 finished with value: inf and parameters: {'hidden_size': 43, 'lstm_layers': 1, 'num_attention_heads': 7, 'dropout': 0.20324537623372685}. Best is trial 0 with value: inf.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


...

`Trainer.fit` stopped: `max_epochs=1` reached.
ValueError: For the given forecasting horizon `n=759`, the provided future covariates at dataset index `0` do not extend far enough into the future. As `n > output_chunk_length` the future covariates must end at time step `2024-07-28 00:00:00`, whereas now they end at time step `2024-01-10 00:00:00`.
[I 2024-09-25 11:07:50,370] Trial 4 finished with value: inf and parameters: {'hidden_size': 26, 'lstm_layers': 3, 'num_attention_heads': 3, 'dropout': 0.3695747225602523}. Best is trial 0 with value: inf.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

   | Name                              | Type                             | Params | Mode 
------------------------------------------------------------------------------------------------
0  | criterion                         | MSELoss                          | 0      | train
1  | train_criterion                   | MSELoss  

Trial failed due to: For the given forecasting horizon `n=759`, the provided future covariates at dataset index `0` do not extend far enough into the future. As `n > output_chunk_length` the future covariates must end at time step `2024-07-28 00:00:00`, whereas now they end at time step `2024-01-10 00:00:00`.
Best hyperparameters:  {'hidden_size': 47, 'lstm_layers': 1, 'num_attention_heads': 1, 'dropout': 0.38824819536107896}


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.
ValueError: For the given forecasting horizon `n=759`, the provided future covariates at dataset index `0` do not extend far enough into the future. As `n > output_chunk_length` the future covariates must end at time step `2024-07-28 00:00:00`, whereas now they end at time step `2024-01-10 00:00:00`.


ValueError: For the given forecasting horizon `n=759`, the provided future covariates at dataset index `0` do not extend far enough into the future. As `n > output_chunk_length` the future covariates must end at time step `2024-07-28 00:00:00`, whereas now they end at time step `2024-01-10 00:00:00`.

In [32]:
print(f"Adjusted future_covariates_test length: {len(future_covariates_test)}")
print(f"Length of series_test after adjustment: {len(series_test)}")


Adjusted future_covariates_test length: 759
Length of series_test after adjustment: 759


In [26]:
print(future_covariates_train.time_index)
print(future_covariates_test.time_index)


DatetimeIndex(['2012-01-01', '2012-01-02', '2012-01-03', '2012-01-04',
               '2012-01-05', '2012-01-06', '2012-01-07', '2012-01-08',
               '2012-01-09', '2012-01-10',
               ...
               '2022-06-21', '2022-06-22', '2022-06-23', '2022-06-24',
               '2022-06-25', '2022-06-26', '2022-06-27', '2022-06-28',
               '2022-06-29', '2022-06-30'],
              dtype='datetime64[ns]', name='Date', length=3834, freq='D')
DatetimeIndex(['2021-12-13', '2021-12-14', '2021-12-15', '2021-12-16',
               '2021-12-17', '2021-12-18', '2021-12-19', '2021-12-20',
               '2021-12-21', '2021-12-22',
               ...
               '2024-07-19', '2024-07-20', '2024-07-21', '2024-07-22',
               '2024-07-23', '2024-07-24', '2024-07-25', '2024-07-26',
               '2024-07-27', '2024-07-28'],
              dtype='datetime64[ns]', name='Date', length=959, freq='D')


In [27]:
print(f"Shape of forecast_scaled: {forecast_scaled.values().shape}")
print(f"Shape of series_test_scaled: {series_test_scaled.values().shape}")


Shape of forecast_scaled: (759, 1)
Shape of series_test_scaled: (759, 1)


In [28]:
print(f"Required start date: {required_start_date}")
print(f"Required end date: {required_end_date}")


Required start date: 2021-12-13 00:00:00
Required end date: 2024-07-28 00:00:00


In [30]:
# Check for NaN values
#if series_test.has_nan_values() or future_covariates_train.has_nan_values():
 #   print("NaN values detected in series_test or future_covariates_train. Please fix the data.")

# Ensure time alignment
print(f"Time alignment check:")
print(f"series_test length: {len(series_test)}, forecast_scaled length: {len(forecast_scaled)}")
print(f"future_covariates_test length: {len(future_covariates_test)}")


Time alignment check:
series_test length: 759, forecast_scaled length: 759
future_covariates_test length: 959


In [33]:
import optuna
from darts.models import TFTModel
from darts import TimeSeries
from darts.models.forecasting.tft_model import QuantileRegression
import pandas as pd
import numpy as np
import torch
import plotly.graph_objs as go
from darts.dataprocessing.transformers import Scaler
from darts.metrics import mape, rmse, mse, mae

# Fixed input chunk length
fixed_input_chunk_length = 200  # You can change this value based on your preference.

# Convert future covariates to TimeSeries objects
future_covariates_train = TimeSeries.from_dataframe(train_df, 'Date', future_covariates_columns).astype('float32')

# Convert full covariates to TimeSeries objects
future_covariates_full = TimeSeries.from_dataframe(df, 'Date', future_covariates_columns, fill_missing_dates=True, freq="D").astype('float32')

# Scaling the data
scaler_series = Scaler()
scaler_covariates = Scaler()

# Fit the scaler on the training data
series_train_scaled = scaler_series.fit_transform(series_train)
future_covariates_train_scaled = scaler_covariates.fit_transform(future_covariates_train)

# Transform the test series using the same scaler
series_test_scaled = scaler_series.transform(series_test)

# Slice future covariates based on series_test's date range to ensure perfect alignment
future_covariates_test = future_covariates_full.slice(
    series_test.start_time(),
    series_test.end_time()
)

# Ensure the future covariates test set is scaled
future_covariates_test_scaled = scaler_covariates.transform(future_covariates_test)

def objective(trial):
    # Suggest hyperparameters (excluding input_chunk_length since it's fixed)
    hidden_size = trial.suggest_int('hidden_size', 8, 64)
    lstm_layers = trial.suggest_int('lstm_layers', 1, 4)
    num_attention_heads = trial.suggest_int('num_attention_heads', 1, 8)
    dropout = trial.suggest_float('dropout', 0.0, 0.5)

    # Initialize the TFT model with suggested hyperparameters
    model = TFTModel(
        input_chunk_length=fixed_input_chunk_length,
        output_chunk_length=len(series_test_scaled),  # Predict over the entire test set length
        hidden_size=hidden_size,
        lstm_layers=lstm_layers,
        num_attention_heads=num_attention_heads,
        dropout=dropout,
        likelihood=QuantileRegression(quantiles=[0.1, 0.5, 0.9]),
        random_state=42,
        add_relative_index=True
    )

    try:
        # Train the model
        model.fit(series_train_scaled, future_covariates=future_covariates_train_scaled, epochs=1, verbose=False)

        # Make predictions
        forecast_scaled = model.predict(n=len(series_test_scaled), future_covariates=future_covariates_test_scaled)

        # Inverse transform the forecast to the original scale
        forecast = scaler_series.inverse_transform(forecast_scaled)

        # Return Mean Squared Error as the objective metric for Optuna to minimize
        error = mse(series_test, forecast)
        if np.isnan(error):
            return float('inf')
        return error
    except Exception as e:
        print(f"Trial failed due to: {e}")
        return float('inf')

# Run the Optuna study with more trials, using the fixed input_chunk_length
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=5, n_jobs=1)

# Print the best hyperparameters
print('Best hyperparameters: ', study.best_params)

# Use the best hyperparameters with the fixed input_chunk_length
best_params = study.best_params

# Initialize the final model with best hyperparameters
best_model = TFTModel(
    input_chunk_length=fixed_input_chunk_length,
    output_chunk_length=len(series_test_scaled),  # Predict over the entire test set length
    hidden_size=best_params['hidden_size'],
    lstm_layers=best_params['lstm_layers'],
    num_attention_heads=best_params['num_attention_heads'],
    dropout=best_params['dropout'],
    likelihood=QuantileRegression(quantiles=[0.1, 0.5, 0.9]),
    random_state=42,
    add_relative_index=True,
    loss_fn=torch.nn.MSELoss()
)

# Train the best model
best_model.fit(series_train_scaled, future_covariates=future_covariates_train_scaled, epochs=1, verbose=True)

# Make predictions starting from the correct point
forecast_scaled = best_model.predict(
    n=len(series_test_scaled),
    future_covariates=future_covariates_test_scaled
)

# Inverse transform the forecast to the original scale
forecast = scaler_series.inverse_transform(forecast_scaled)

# Convert TimeSeries to DataFrame for Plotly plotting
test_df_plotly = series_test.pd_dataframe()
forecast_df_plotly = forecast.pd_dataframe()

# Plot the results using Plotly
fig = go.Figure()

# Add actual test data trace
fig.add_trace(go.Scatter(x=test_df_plotly.index, y=test_df_plotly['Day_ahead_price (€/MWh)'],
                         mode='lines', name='Actual Test Data', line=dict(color='darkblue')))

# Add forecast data trace
fig.add_trace(go.Scatter(x=forecast_df_plotly.index, y=forecast_df_plotly['Day_ahead_price (€/MWh)'],
                         mode='lines', name='TFT Forecast on Test Data', line=dict(color='red')))

# Update layout
fig.update_layout(
    title='TFT Model - Test Performance with Optuna Hyperparameter Tuning',
    xaxis_title='Date',
    yaxis_title='Day Ahead Price (€/MWh)',
    legend=dict(
        x=1,
        y=1,
        xanchor='right',
        yanchor='top',
        bordercolor='black',
        borderwidth=1
    ),
    template='plotly'
)

# Show the plot
fig.show()

# Evaluate the model using Darts' metrics
print(f'Mean Absolute Error on Test Set: {mae(series_test, forecast)}')
print(f'Mean Absolute Percentage Error on Test Set: {mape(series_test, forecast)}')
print(f'Mean Squared Error on Test Set: {mse(series_test, forecast)}')
print(f'Root Mean Squared Error on Test Set: {rmse(series_test, forecast)}')


[I 2024-09-26 10:57:17,038] A new study created in memory with name: no-name-82c149ec-b67a-4554-8500-e1aa45f0cbda
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer.fit` stopped: `max_epochs=1` reached.
ValueError: For the given forecasting case, the provided future covariates at dataset index `0` do not extend far enough into the past. The future covariates must start at time step `2021-12-13 00:00:00`, whereas now they start at time step `2022-07-01 00:00:00`.
[I 2024-09-26 10:58:51,245] Trial 0 finished with value: inf and parameters: {'hidden_size': 26, 'lstm_layers': 3, 'num_attention_heads': 7, 'dropout': 0.3097981299534905}. Best is trial 0 with value: inf.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Trial failed due to: For the given forecasting case, the provided future covariates at dataset index `0` do not extend far enough into the past. The future covariates must start at time step `2021-12-13 00:00:00`, whereas now they start at time step `2022-07-01 00:00:00`.


`Trainer.fit` stopped: `max_epochs=1` reached.
ValueError: For the given forecasting case, the provided future covariates at dataset index `0` do not extend far enough into the past. The future covariates must start at time step `2021-12-13 00:00:00`, whereas now they start at time step `2022-07-01 00:00:00`.
[I 2024-09-26 11:00:42,041] Trial 1 finished with value: inf and parameters: {'hidden_size': 47, 'lstm_layers': 2, 'num_attention_heads': 5, 'dropout': 0.08471606478411381}. Best is trial 0 with value: inf.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Trial failed due to: For the given forecasting case, the provided future covariates at dataset index `0` do not extend far enough into the past. The future covariates must start at time step `2021-12-13 00:00:00`, whereas now they start at time step `2022-07-01 00:00:00`.


`Trainer.fit` stopped: `max_epochs=1` reached.
ValueError: For the given forecasting case, the provided future covariates at dataset index `0` do not extend far enough into the past. The future covariates must start at time step `2021-12-13 00:00:00`, whereas now they start at time step `2022-07-01 00:00:00`.
[I 2024-09-26 11:02:47,035] Trial 2 finished with value: inf and parameters: {'hidden_size': 57, 'lstm_layers': 4, 'num_attention_heads': 6, 'dropout': 0.24911270467868285}. Best is trial 0 with value: inf.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


...

Trial failed due to: For the given forecasting case, the provided future covariates at dataset index `0` do not extend far enough into the past. The future covariates must start at time step `2021-12-13 00:00:00`, whereas now they start at time step `2022-07-01 00:00:00`.
Best hyperparameters:  {'hidden_size': 26, 'lstm_layers': 3, 'num_attention_heads': 7, 'dropout': 0.3097981299534905}


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

   | Name                              | Type                             | Params | Mode 
------------------------------------------------------------------------------------------------
0  | criterion                         | MSELoss                          | 0      | train
1  | train_criterion                   | MSELoss                          | 0      | train
2  | val_criterion                     | MSELoss                          | 0      | train
3  | train_metrics                     | MetricCollection                 | 0      | train
4  | val_metrics                       | MetricCollection                 | 0      | train
5  | input_embeddings                  | _MultiEmbedding                  | 0      | train
6  | static_covariates_vsn             | _VariableSelectionNetwork        | 0      | train
7  | encoder_vsn                       | _VariableSelection

Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.
ValueError: For the given forecasting case, the provided future covariates at dataset index `0` do not extend far enough into the past. The future covariates must start at time step `2021-12-13 00:00:00`, whereas now they start at time step `2022-07-01 00:00:00`.


ValueError: For the given forecasting case, the provided future covariates at dataset index `0` do not extend far enough into the past. The future covariates must start at time step `2021-12-13 00:00:00`, whereas now they start at time step `2022-07-01 00:00:00`.